In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from PIL import Image

### 1. Rename images and get their paths

In [ ]:
# Define a function to rename image paths
def rename_files(base_dir):

  image_dir, label_dir = os.path.join(base_dir, 'images'), os.path.join(base_dir, 'labels')
  image_filenames = os.listdir(image_dir)
  image_filenames.sort()

  for idx, image_filename in enumerate(image_filenames):
    os.rename(os.path.join(image_dir, image_filename), os.path.join(image_dir, str(idx+1) + '.jpg'))
    os.rename(os.path.join(label_dir, image_filename)[:-3]+'txt', os.path.join(label_dir, str(idx+1) + '.txt'))

# Rename image paths
rename_files('data/train/')
rename_files('data/valid/')


In [ ]:
# Define a function to get image paths
def get_image_paths(image_directory):
  """
  Aggregates image filepaths from given image_directory path.

  Args:
    image_directory (str): path to the image directory

  Returns:
    image_paths: a list containing filepaths of the
                 images in the specified directory path
  """

  image_paths = []
  image_filenames = os.listdir(image_directory)
  image_paths = [os.path.join(image_directory, image_filename) for image_filename in image_filenames]

  return image_paths

# Get image paths of explore1 video
images_train = get_image_paths('data/train/images')
images_valid = get_image_paths('data/valid/images')
images_train[:2], images_valid[:2]

(['data/train/images/1.jpg', 'data/train/images/47.jpg'],
 ['data/valid/images/1.jpg', 'data/valid/images/2.jpg'])

### 2. Train YOLOv5s

In [ ]:
# Train YOLOv5s
!python train.py --img 640 --batch 32 --epochs 500 --data data.yaml --weights yolov5s.pt --cache --device 0

requirements: Ultralytics requirement "gitpython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 272.2 MB/s eta 0:00:00

requirements: 1 package updated per ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, 

### 3. Load Model

In [ ]:
weights_path = 'yolov5/runs/train/exp4/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)

# Configure model
model.conf = 0.4  # NMS confidence threshold
model.iou = 0.5  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
model.max_det = 10  # maximum number of detections per image
model.amp = False  # Automatic Mixed Precision (AMP) inference

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-6-4 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


### 4. Make Inference

In [ ]:
# Inference
results_01 = model(images_train)
results_02 = model(images_valid)

# Results
results_01.save()
results_02.save()


Saved 465 images to runs/detect/exp8
Saved 84 images to runs/detect/exp9


In [ ]:
results_02.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,288.282532,247.778076,392.450684,379.143677,0.922690,2,Docking Station
1,602.877930,0.000000,640.000000,155.107758,0.692596,0,Control Panel


In [ ]:
results_02.show()

Output hidden; open in https://colab.research.google.com to view.